In [47]:
from bs4 import BeautifulSoup
import requests

url = "https://lihai.blog.csdn.net/article/details/139259946"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
    'Cookie': 'uuid_tt_dd=10_10074599950-1750400284399-202786; fid=20_32036529762-1750400285360-529188; p_uid=U010000; csdn_newcert_qq_46796425=1; c_ab_test=1; c_dl_prid=-; c_dl_rid=1751890689391_995661; c_dl_fref=https://so.csdn.net/so/search; c_dl_fpage=/download/YOULANSHENGMENG/85720217; c_dl_um=-; c_segment=2; dc_sid=221b3972b67aa23e892887b9262e268e; Hm_lvt_6bcd52f51e9b3dce32bec4a3997715ac=1751787040,1752538600,1752756401,1753271085; HMACCOUNT=06B79B813DE53644; __gads=ID=5299f7838bc11b36:T=1750426964:RT=1753455568:S=ALNI_MZxA42ueUPNqDqRpPFcmBcqCOxRnw; __gpi=UID=00001135239948d3:T=1750426964:RT=1753455568:S=ALNI_MY3I-1XiCy-8oNgQbcwbJ-f8jsMtA; __eoi=ID=84f67d1f5eb16c3b:T=1750426964:RT=1753455568:S=AA-Afjb7y_bY2RXkzj69RT7HLPqL; FCNEC=%5B%5B%22AKsRol9HSUorfTByDPEZkEVtR3HM-wrZCq2sjAHHoMK9oOe1Ix_TS45NudwcTR03oY_Pdo_ISnEWy26UL-JxMVzvFrMgp4Dd8H0C1MbQECS_xVSvmHj1eqNKS5sNJfmQa4-1PhOCYzyZI-tp9RJMGaRjYXrLx8I8xQ%3D%3D%22%5D%5D; c_first_ref=default; c_utm_source=BWXQ_bottombtn; _tea_utm_cache_586864={%22utm_source%22:%22BWXQ_bottombtn%22}; c_first_page=https%3A//blog.csdn.net/wangpaifeixingyuan/category_12967381.html; SESSION=2eb69f27-7912-477e-9840-48f5edd515cb; hide_login=1; loginbox_strategy=%7B%22taskId%22%3A317%2C%22abCheckTime%22%3A1753537198423%2C%22version%22%3A%22ExpA%22%2C%22nickName%22%3A%22qq_46796425%22%2C%22blog-threeH-dialog-expa%22%3A1753537336406%7D; creative_btn_mp=2; UserName=qq_58544736; UserInfo=c28c609d425749cb8561a53e1a435575; UserToken=c28c609d425749cb8561a53e1a435575; UserNick=qq_58544736; AU=91A; UN=qq_58544736; BT=1753537369874; csdn_newcert_qq_58544736=1; c_pref=https%3A//blog.csdn.net/qq_58544736%3Ftype%3Dask; c_ref=https%3A//blog.csdn.net/wangpaifeixingyuan/category_12680585.html; _clck=15rygjy%7C2%7Cfxy%7C0%7C1997; dc_session_id=10_1753593980750.787079; c_dsid=11_1753593980585.970692; is_advert=1; bc_bot_session=17535939826400d38be4e74374; c_page_id=default; log_Id_pv=2; Hm_lpvt_6bcd52f51e9b3dce32bec4a3997715ac=1753594000; _clsk=1k6cx5x%7C1753594001392%7C4%7C0%7Cj.clarity.ms%2Fcollect; log_Id_view=106; dc_tos=t01m51; dc_tos=t01m6u'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

In [48]:
# 1. 提取标题
title_tag = soup.find("h1", class_="title-article", id="articleContentId")
title_text = title_tag.get_text(strip=True) if title_tag else "未找到标题"

# 把第一个“例”字去掉（只去第一个，不影响后面数字）
title_text = title_text.replace("例", "", 1)

print(title_text)
# 2. 提取所有代码块（保持原样）
code_blocks = []

for pre_tag in soup.find_all("pre"):
    code_tag = pre_tag.find("code")
    if code_tag:
        code_text = code_tag.get_text()
        if code_text.strip():
            code_blocks.append(code_text)
print(f"共找到 {len(code_blocks)} 个代码块")

# 3. 提取“代码解析”部分
target_h2 = None
for h2 in soup.find_all("h2"):
    if "代码解析" in h2.get_text():
        target_h2 = h2
        break

code_analysis_lines = []

if target_h2:
    extracted = []
    for sibling in target_h2.find_next_siblings():
        if sibling.name == "h2":
            break
        extracted.append(sibling)

    code_analysis_lines.append("#### 代码解析\n")

    for tag in extracted:
        if tag.name == "p":
            lines = tag.get_text(strip=True).splitlines()
            for line in lines:
                code_analysis_lines.append(f"> {line.strip()}")
            code_analysis_lines.append(">")
        elif tag.name == "ol":
            for i, li in enumerate(tag.find_all("li"), 1):
                code_analysis_lines.append(f"> {i}. {li.get_text(strip=True)}")
            code_analysis_lines.append(">")
        elif tag.name in ["pre", "code"]:
            code_text = tag.get_text()
            if code_text.strip():
                code_analysis_lines.append("> ```cpp")
                for line in code_text.splitlines():
                    code_analysis_lines.append(f"> {line}")
                code_analysis_lines.append("> ```")
                code_analysis_lines.append(">")

else:
    code_analysis_lines.append("#### 代码解析\n> 未找到解析内容\n")

# 4. 写入 article_full.md 文件
with open(title_text+".md", "w", encoding="utf-8") as f:
    # 标题
    f.write(f"### {title_text}\n\n")

    # 代码部分
    f.write("#### 代码\n\n")
    for i, code in enumerate(code_blocks, 1):
        f.write("```cpp\n")
        f.write(code)
        f.write("\n```\n\n")

    # 代码解析部分
    for line in code_analysis_lines:
        f.write(line + "\n")

print("✅ 所有内容已按要求保存到" + title_text + ".md")

【0374】ask view z dir 查询视图Z轴方向
共找到 1 个代码块
✅ 所有内容已按要求保存到【0374】ask view z dir 查询视图Z轴方向.md
